In [ ]:
 # Vitaly Marin version 2 with early stopping upon reaching covergence close to -100 on this game within 5% or so 


from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# path to project

In [ ]:
PATH = 'drive/MyDrive/CS7648_group7_project/'

In [ ]:
%cd drive/MyDrive/IRL_DQfD
# !pwd

/content/drive/MyDrive/IRL_DQfD


# Imports and installations

In [ ]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 7.7 MB/s 
     |████████████████████████████████| 168 kB 81.3 MB/s 
     |████████████████████████████████| 182 kB 71.9 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 79.8 MB/s 
     |████████████████████████████████| 166 kB 80.0 MB/s 
     |████████████████████████████████| 166 kB 82.4 MB/s 
     |████████████████████████████████| 162 kB 79.2 MB/s 
     |████████████████████████████████| 162 kB 78.7 MB/s 
     |████████████████████████████████| 158 kB 77.6 MB/s 
     |████████████████████████████████| 157 kB 82.2 MB/s 
     |████████████████████████████████| 157 kB 74.0 MB/s 
     |████████████████████████████████| 157 kB 82.4 MB/s 
     |████████████████████████████████| 157 kB 86.5 MB/s 
     |████████████████████████████████| 157 kB 81.2 MB/s 
     |████████████████████████████████| 157 kB 86.8 MB/s 
     |████████████████████████████████| 157 kB 84.4 MB/s 
     |███████████

In [ ]:
import wandb
wandb.login()
#6667acf0131f4abd55b0c07404f6f71f48cf2379

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
import numpy as np
import random
from collections import deque
import pickle
import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
# from tensorflow.keras.optimizers import Adam

In [ ]:
!pip install gym[atari,accept-rom-license]==0.21.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 6.5 MB/s 
     |████████████████████████████████| 1.6 MB 53.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616822 sha256=91df20d97866540a2201f063f6dafb9c9323da6739b3cb8e1f5b54885c7e6dd5
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441028 sha256=0b335aebe17c54b8c9859f00a88432d808b793ab32d8ab6ff77be4c2724c5bab
  Stored in directory: /root/.cache/pip/wheels/51/08/c5/28b973078691a3f8baf99fcaec1ed8f0e05ef6e54d2390212c
Successfully built gym AutoROM.accept-rom-license
  Attempting uninstall: gym
    Found existing installatio

In [ ]:
#https://www.gymlibrary.dev/environments/classic_control/
import gym
env = gym.make('Acrobot-v1')


# Network

In [ ]:
from typing import Set
# Making tf as torch

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self,input, output):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input, 64)
    self.fc2 = nn.Linear(64, 64*2)
    self.fc3 = nn.Linear(64*2, output)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

class DQN_Torch:
    def __init__(self, state_size, action_size):
        
        #define the state size
        self.state_size = state_size
        
        #define the action size
        self.action_size = action_size
        
        #define the replay buffer
        self.replay_buffer = deque(maxlen=10000)

        #define the demo buffer
        self.demo_buffer = deque(maxlen=10000)
        
        #define the discount factor
        self.gamma = 0.9  
        
        #define the epsilon value
        self.epsilon = 0.01
        
        #define the update rate at which we want to update the target network
        self.update_rate = 20
        
        #define the main network
        self.main_network = Net(self.state_size, self.action_size)
        
        self.loss_fn = nn.MSELoss()
        self.loss_supervised_fn = nn.L1Loss()

        self.optim = torch.optim.Adam(self.main_network.parameters(),lr=0.001)
      
        #define the target network
        self.target_network = Net(self.state_size, self.action_size)
        self.target_network.requires_grad = False

        # self.device = torch.device('cpu')
        if torch.cuda.is_available():
          print("Using CUDA...")
          self.device = torch.device('cuda')
          self.main_network = self.main_network.to(device)
          self.target_network = self.target_network.to(device)
        else:
          print("Using CPU...")
          self.device = torch.device('cpu')
        
        #copy the weights of the main network to the target network
        self.target_network.load_state_dict(self.main_network.state_dict())

    #We learned that we train DQN by randomly sampling a minibatch of transitions from the
    #replay buffer. So, we define a function called store_transition which stores the transition information
    #into the replay buffer

    def store_transistion(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))
        
    # Save expert demonstrations
    def store_demonstrations(self, state, action, reward, next_state, done):
        self.demo_buffer.append((state, action, reward, next_state, done))

    #We learned that in DQN, to take care of exploration-exploitation trade off, we select action
    #using the epsilon-greedy policy. So, now we define the function called epsilon_greedy
    #for selecting action using the epsilon-greedy policy.
    
    def epsilon_greedy(self, state):
        if random.uniform(0,1) < self.epsilon:
            return np.random.randint(self.action_size)
        
        Q_values = self.main_network(torch.from_numpy(state).type(torch.float).to(device))
        
        # print(Q_values, torch.argmax(Q_values[0]).item())
        return torch.argmax(Q_values).item()

    # Pretrain using expert demonstrations
    def pre_train(self):
      print('Pre-training ...')

      ## Loading the pre-trained network
      # pretrained_state_dict = pickle.load(open(PATH+'project/Sai_folder/code/experiment_runtime_log/acrobot2_pre_trained_network_400_100_256', 'rb'))
      # self.main_network.load_state_dict(pretrained_state_dict)
      # self.target_network.load_state_dict(pretrained_state_dict)

      ## Pre-training the network
      for i in range(200): ## PARAM_CHECK: how many steps for pretraining?
          self.train(batch_size = 256, pretrain=True)
          if i % 10 == 0 and i > 0:
              get_demonstrations_from_trained_dqn(self.main_network.state_dict(), store=False)
              print('{}th step of pre-train finish ...'.format(i))

      print('All pre-train finish.')

    
    # Supervised Loss function for the demo buffer data

    def loss_supervised(self, X, y):
        X_max, _ = torch.max(X, axis=1)
        # print(X_max, y)
        return self.loss_supervised_fn(X_max, y)


    #train the network
    def train(self, batch_size = 32, pretrain = False, sample_demo = False, demo_sample_percent = 0.25, supervised_weight = 1.0):
        
        data_buffer = []
        demo_buffer = []

        # # data_buffer = self.demo_buffer if pretrain else self.replay_buffer
        if pretrain:
          #sample a mini batch of transition from the demo buffer if pretrain
          demo_buffer = random.sample(self.demo_buffer, batch_size)
        elif sample_demo:
          ## Add some of the expert_demos to the replay_buffer
          demo_sample_size = int(batch_size * demo_sample_percent)
          demo_buffer = random.sample(self.demo_buffer, demo_sample_size)
          data_buffer = random.sample(self.replay_buffer, batch_size - demo_sample_size)
          # print(f'Demos: {len(self.demo_buffer)}, Data: {len(data_buffer)}') 
        else:
          ## Don't include any expert demonstrations during training
          data_buffer = random.sample(self.replay_buffer, batch_size)
          
        # minibatch = random.sample(self.replay_buffer, batch_size)
        # minibatch = random.sample(data_buffer, batch_size)

        minibatch = []
        if(len(data_buffer)!=0):
          minibatch.append(data_buffer)
        if(len(demo_buffer)!= 0):
          minibatch.append(demo_buffer)
        minibatch = np.concatenate(minibatch, axis=0)

        X = [] # State 
        y = [] # Expected Q values of the input state
        
        #compute the Q value using the target network 
        for state, action, reward, next_state, done in minibatch:
            if not done:
                target_Q = (reward + self.gamma * torch.max(self.target_network(torch.from_numpy(next_state).type(torch.float).to(device))).detach().cpu().numpy())
            else:
                target_Q = reward
                
            #compute the Q value using the main network
            state = torch.from_numpy(state).type(torch.float).to(self.device) 
            Q_values = self.main_network(state)

            # print(Q_values)
            Q_values[action] = target_Q
            # print(Q_values)
            X.append(state)
            y.append(Q_values)
        
        X = torch.squeeze(torch.stack(X, 0), 1)
        y = torch.squeeze(torch.stack(y, 0), 1)
        # X = torch.from_numpy(np.array(X).astype('float32')).type(torch.float).to(self.device)
        # y = torch.from_numpy(np.array(y).astype('float32')).type(torch.float).to(self.device)

        y_pred = self.main_network(X)

        X_supervised = [] # Q_value of the state with masks
        y_supervised = [] # Actual Q_value of the selected action

        for state, action, reward, next_state, done in demo_buffer:
            #compute the Q value using the main network
            state = torch.from_numpy(state).type(torch.float).to(self.device) 
            Q_values = self.main_network(state)

            mask_action = 0.8 * torch.ones(self.action_size).type(torch.float).to(self.device)
            mask_action[action] = 0

            X_supervised.append(Q_values + mask_action)
            y_supervised.append(Q_values[action])
        
        if(len(demo_buffer) > 0):
          # print(X_supervised, y_supervised)
          X_supervised = torch.squeeze(torch.stack(X_supervised, 0), 1)
          y_supervised = torch.stack(y_supervised, 0).detach()

        if(len(demo_buffer) > 0):
          loss = self.loss_fn(y_pred, y) + supervised_weight * self.loss_supervised(X_supervised, y_supervised) # TODO: lambda is the weighting parameter for the supervised loss. Value = ?
        else:
          loss = self.loss_fn(y_pred, y)

        self.optim.zero_grad()
        loss.backward()
        self.optim.step()
            
    #update the target network weights by copying from the main network
    def update_target_network(self):
        self.target_network.load_state_dict(self.main_network.state_dict())

# Hyperparameters

In [ ]:
num_episodes = 200
num_timesteps = 1000
batch_size = 64
num_screens = 4

#Game parameters
state_size = 6
action_size = 3

max_reward_goal = -100*1.05


# Get expert demonstrations

In [ ]:
def get_demonstrations_from_trained_dqn(trained_state_dict, store=False):
  trained_dqn = DQN_Torch(state_size, action_size)
  trained_dqn.target_network.load_state_dict(trained_state_dict)
  trained_dqn.main_network.load_state_dict(trained_state_dict)
  # new_action = trained_dqn.epsilon_greedy()
  done = False
  avg_reward = 0

  for i in range(100):
      
      #set return to 0
      Return = 0
      
      #preprocess the game screen
      state = env.reset()

      #for each step in the episode
      for t in range(1000):

          action = trained_dqn.epsilon_greedy(state)

          #perform the selected action
          next_state, reward, done, _ = env.step(action)

          #store the transition information
          trained_dqn.store_transistion(state, action, reward, next_state, done)

          #update current state to next state
          state = next_state

          #update the return
          Return += reward

          if done:
            # print(f"Iteration {i}: Time Steps Completed: {t}")
            avg_reward += Return
            break

  print(f"Total Avg Reward = {avg_reward/100}")
  # print(trained_dqn.replay_buffer)
  if store:
    pickle.dump(trained_dqn.replay_buffer, open('acrobot2_expert_demonstrations', 'wb'))

In [ ]:
# trained_state_dict = pickle.load(open(PATH+'project/Sai_folder/code/model_weights/model_weights_400.pkl', 'rb'))
# get_demonstrations_from_trained_dqn(trained_state_dict, store=True)

Total Avg Reward = 148.76


# DDQN

In [ ]:
def run_DDQN(dqn):

  done = False
  time_step = 0
  ep_reward = []
  ep_steps = []

  wandb.init(
      entity="irl_team7",
      # Set the project where this run will be logged
      project="DQfD", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name = f"acrobot2_ddqn_{num_episodes}_{batch_size}",
      # name=f"cartpole_{anneal_type}_{decay_rate}", 
      # Track hyperparameters and run metadata
      config={
          "environment": "Acrobot",
          "network": "DDQN",
          "num_episodes": num_episodes,
          "num_timesteps": num_timesteps,
          "batch_size": batch_size,
          "num_screens": num_screens,
          "state_size": state_size,
          "action_size": action_size
          # "random_seed": seed_val
      })

  converged_model_true = False 

  #for each episode
  for i in range(num_episodes):
      
      #set return to 0
      Return = 0
      
      #preprocess the game screen
      state = env.reset()

      #for each step in the episode
      for t in range(num_timesteps):
          #render the environment
          # env.render(render_mode = "human")
          
          #update the time step
          time_step += 1
          
          #update the target network
          if time_step % dqn.update_rate == 0:
              dqn.update_target_network()

          #select the action
          action = dqn.epsilon_greedy(state)

          #perform the selected action
          next_state, reward, done, _ = env.step(action)

          #store the transition information
          dqn.store_transistion(state, action, reward, next_state, done)

          #update current state to next state
          state = next_state

          #update the return
          Return += reward

          # if t%1000 == 0:
          #   print(f"Episode: {i}  TimeStep: {t}  Cum Reward: {Return}")

          #if the episode is done then print the return
          if done:
              print(f"Episode: {i} Time Step: {t} Cumulative Rewards: {Return}")
              ep_reward.append(Return)
              ep_steps.append(t)
              break
              
          #if the number of transistions in the replay buffer is greater than batch size
          #then train the network
          if len(dqn.replay_buffer) > batch_size:
              dqn.train(batch_size)

      wandb.log({"rewards": Return, "time_step": t})
      if(Return >= max_reward_goal):
         converged_model_true = True 

      if (converged_model_true or (i+1)%50 == 0):
          pickle.dump(dqn.main_network.state_dict(), open(f'acrobot2_ddqn_weights_{i+1}', 'wb')) # save model every 50 epochs 
          if(converged_model_true ):
              break

  wandb.finish()

  return ep_reward

#DQfD

In [ ]:
def run_DQfD(dqn, anneal_type = "constant", initial_sample_percent = 0, anneal_threshold = None, decay_rate = 0.9, expo_const = 0.5, seed_val=1):
  ## anneal_type = ["constant", "linear", "exponential", "threshold"]

  ## Pre-train the network on the expert demonstration

  # get the demonstrations (Assuming there is a file)
  # expert_demos = pickle.load(open('expert_demonstrations', 'rb'))
  random.seed(seed_val)
  np.random.seed(seed_val)

  expert_demos = pickle.load(open('acrobot_expert_demonstrations', 'rb'))


  dqn.demo_buffer = expert_demos

  print(f"Number of <state,action> pairs in the demonstrator data = {len(expert_demos)}")

  if anneal_type == "constant":
    run_name = f"acrobot_constant_{initial_sample_percent}_{seed_val}"
  elif anneal_type == "linear":
    run_name = f"acrobot_linear_{initial_sample_percent}_{decay_rate}_{seed_val}"
  elif anneal_type == "exponential":
    run_name = f"acrobot_exponential_{initial_sample_percent}_{expo_const}_{seed_val}"
  elif anneal_type == "threshold":
    run_name = f"acrobot_threshold_{initial_sample_percent}_{anneal_threshold}_{seed_val}"

  wandb.init(
      entity="irl_team7",
      # Set the project where this run will be logged
      project="DQfD", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name = run_name,
      # name=f"cartpole_{anneal_type}_{decay_rate}", 
      # Track hyperparameters and run metadata
      config={
          "environment": "Acrobot",
          "num_episodes": num_episodes,
          "num_timesteps": num_timesteps,
          "batch_size": batch_size,
          "num_screens": num_screens,
          "state_size": state_size,
          "action_size": action_size,
          "initial_sample_percent": initial_sample_percent, 
          "anneal_type": anneal_type,
          "decay_rate": decay_rate,
          "expo_const": expo_const,
          "anneal_threshold": anneal_threshold,
          "random_seed": seed_val
      })


  ## Pre-train the network
  dqn.pre_train()

  done = False
  time_step = 0
  ep_reward = []
  ep_steps = []


  # if anneal_type == "constant":
  #   demo_sample_percent = initial_sample_percent
  # else:
  #   demo_sample_percent = 1    ## Start with all expert demonstrations and gradually decrease


  #for each episode
  for i in range(num_episodes):
      
      #set return to 0
      Return = 0
      
      #preprocess the game screen
      state = env.reset()

      if anneal_type == "constant":
        demo_sample_percent = initial_sample_percent
      elif anneal_type == "linear":
        # demo_sample_percent = decay_rate * demo_sample_percent
        demo_sample_percent = max(0, initial_sample_percent - decay_rate * i)
      elif anneal_type == "exponential":
        demo_sample_percent = initial_sample_percent * np.exp(-expo_const * i)
      elif anneal_type == "threshold":
        demo_sample_percent = initial_sample_percent
        if len(ep_reward) > 10 and sum(ep_reward[-10:])/10 > anneal_threshold:
          demo_sample_percent = 0.0

      print(f'Annealing: {anneal_type}, Demo Sampling Percent: {demo_sample_percent}')

      #for each step in the episode
      for t in range(num_timesteps):
          #render the environment
          # env.render(render_mode = "human")
          
          #update the time step
          time_step += 1
          
          #update the target network
          if time_step % dqn.update_rate == 0:
              dqn.update_target_network()

          #select the action
          action = dqn.epsilon_greedy(state)

          #perform the selected action
          next_state, reward, done, _ = env.step(action)

          #store the transition information
          dqn.store_transistion(state, action, reward, next_state, done)

          #update current state to next state
          state = next_state

          #update the return
          Return += reward

          # if t%1000 == 0:
          #   print(f"Episode: {i}  TimeStep: {t}  Cum Reward: {Return}")

          #if the episode is done then print the return
          if done:
              print(f"Episode: {i} Time Step: {t} Cumulative Rewards: {Return}")
              ep_reward.append(Return)
              ep_steps.append(t)
              break
              
          #if the number of transistions in the replay buffer is greater than batch size
          #then train the network
          if len(dqn.replay_buffer) > batch_size:
              dqn.train(batch_size, sample_demo = True, supervised_weight = 0.1, demo_sample_percent = demo_sample_percent)
    
      if(len(ep_reward) > 5 and np.mean(ep_reward[-5:0])==500):
        break

      wandb.log({"rewards": Return, "time_step": t})
  
  wandb.finish()

  pickle.dump(ep_reward, open(run_name, 'wb'))

  return ep_reward

# Display

In [ ]:
def plot_rewards(ep_reward):
  plt.plot(np.arange(len(ep_reward)), ep_reward)
  plt.show()

# Training DDQN

In [ ]:
dqn = DQN_Torch(state_size, action_size)
run_DDQN(dqn)

Using CUDA...


wandb: Currently logged in as: kmittal37 (irl_team7). Use `wandb login --relogin` to force relogin


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Episode: 0 Time Step: 499 Cumulative Rewards: -500.0
Episode: 1 Time Step: 499 Cumulative Rewards: -500.0
Episode: 2 Time Step: 499 Cumulative Rewards: -500.0
Episode: 3 Time Step: 133 Cumulative Rewards: -133.0
Episode: 4 Time Step: 125 Cumulative Rewards: -125.0
Episode: 5 Time Step: 140 Cumulative Rewards: -140.0
Episode: 6 Time Step: 295 Cumulative Rewards: -295.0
Episode: 7 Time Step: 304 Cumulative Rewards: -304.0
Episode: 8 Time Step: 440 Cumulative Rewards: -440.0
Episode: 9 Time Step: 499 Cumulative Rewards: -500.0
Episode: 10 Time Step: 499 Cumulative Rewards: -500.0
Episode: 11 Time Step: 310 Cumulative Rewards: -310.0
Episode: 12 Time Step: 454 Cumulative Rewards: -454.0
Episode: 13 Time Step: 279 Cumulative Rewards: -279.0
Episode: 14 Time Step: 248 Cumulative Rewards: -248.0
Episode: 15 Time Step: 301 Cumulative Rewards: -301.0
Episode: 16 Time Step: 259 Cumulative Rewards: -259.0
Episode: 17 Time Step: 454 Cumulative Rewards: -454.0
Episode: 18 Time Step: 367 Cumulative 

rewards,▇▄▃▇▄▆▇▇▅▆▅▅▆▇█▆▅▆▅▃▅▆▄▆▆▄▇▆▅▁▃▄▆▆▇▅▄▅▄▆
time_step,▂▅▆▂▅▃▂▂▄▃▄▄▃▂▁▃▄▃▄▆▄▃▅▃▃▅▂▃▄█▆▅▃▃▂▄▅▄▅▃
rewards,-222.0
time_step,222


[-500.0,
 -500.0,
 -500.0,
 -133.0,
 -125.0,
 -140.0,
 -295.0,
 -304.0,
 -440.0,
 -500.0,
 -500.0,
 -310.0,
 -454.0,
 -279.0,
 -248.0,
 -301.0,
 -259.0,
 -454.0,
 -367.0,
 -181.0,
 -271.0,
 -500.0,
 -227.0,
 -391.0,
 -195.0,
 -359.0,
 -302.0,
 -500.0,
 -216.0,
 -274.0,
 -323.0,
 -140.0,
 -194.0,
 -198.0,
 -179.0,
 -197.0,
 -148.0,
 -214.0,
 -155.0,
 -164.0,
 -143.0,
 -197.0,
 -468.0,
 -344.0,
 -154.0,
 -242.0,
 -210.0,
 -162.0,
 -205.0,
 -228.0,
 -238.0,
 -207.0,
 -182.0,
 -246.0,
 -442.0,
 -249.0,
 -211.0,
 -263.0,
 -460.0,
 -117.0,
 -146.0,
 -131.0,
 -316.0,
 -187.0,
 -256.0,
 -368.0,
 -204.0,
 -396.0,
 -380.0,
 -229.0,
 -267.0,
 -171.0,
 -252.0,
 -259.0,
 -197.0,
 -340.0,
 -189.0,
 -304.0,
 -266.0,
 -244.0,
 -123.0,
 -179.0,
 -188.0,
 -260.0,
 -342.0,
 -273.0,
 -179.0,
 -295.0,
 -169.0,
 -192.0,
 -133.0,
 -196.0,
 -418.0,
 -255.0,
 -333.0,
 -227.0,
 -229.0,
 -233.0,
 -264.0,
 -407.0,
 -233.0,
 -350.0,
 -147.0,
 -297.0,
 -500.0,
 -286.0,
 -500.0,
 -216.0,
 -173.0,
 -424.0,
 -185.0,
 

# Trying different annealing

In [ ]:
# dqn = DQN_Torch(state_size, action_size)

# expert_demos = pickle.load(open(PATH+'project/Sai_folder/code/demo_data/expert_demonstrations', 'rb'))
# dqn.demo_buffer = expert_demos

# dqn.pre_train()

# # DDQN at the end of 400 iterations, 100 epochs with batch size 256 for pre training
# pickle.dump(dqn.main_network.state_dict(), open(PATH+'project/Sai_folder/code/experiment_runtime_log/pre_trained_network_400_100_256', 'wb'))

Pre-training ...
Total Avg Reward = 9.34
10th step of pre-train finish ...
Total Avg Reward = 12.3
20th step of pre-train finish ...
Total Avg Reward = 19.44
30th step of pre-train finish ...
Total Avg Reward = 22.06
40th step of pre-train finish ...
Total Avg Reward = 29.1
50th step of pre-train finish ...
Total Avg Reward = 34.03
60th step of pre-train finish ...
Total Avg Reward = 35.62
70th step of pre-train finish ...
Total Avg Reward = 40.36
80th step of pre-train finish ...
Total Avg Reward = 46.64
90th step of pre-train finish ...
All pre-train finish.


In [ ]:
for k in [1]: ## k is the seed value
  # Annealing Type - Exponential
  for sample_rate in [0.25, 0.5]:
    for ec in [0.001, 0.01, 0.1]:
      run_DQfD(DQN_Torch(state_size, action_size), anneal_type="exponential", initial_sample_percent = float(sample_rate), expo_const = float(ec), seed_val = int(k))

  # Annealing Type - Constant
  for sample_rate in [0, 0.25, 0.5, 0.75, 1.0]:
    run_DQfD(DQN_Torch(state_size, action_size), anneal_type="constant", initial_sample_percent = float(sample_rate), seed_val = int(k))

  # Annealing Type - Linear
  for sample_rate in [0.25, 0.5]:
    for dr in [0.01, 0.005, 0.0025]:
      run_DQfD(DQN_Torch(state_size, action_size), anneal_type="linear", initial_sample_percent = float(sample_rate), decay_rate = float(dr), seed_val = int(k))

  # Annealing Type - Threshold
  for sample_rate in [0.25, 0.5]:
    for reward_threshold in [100, 200, 300]:
      run_DQfD(DQN_Torch(state_size, action_size), anneal_type="threshold", initial_sample_percent = float(sample_rate), anneal_threshold = float(reward_threshold), seed_val = int(k))

Using CUDA...


wandb: Currently logged in as: saiprasath3344 (irl_team7). Use `wandb login --relogin` to force relogin


Number of <state,action> pairs in the demonstrator data = 10000


Pre-training ...
All pre-train finish.
Annealing: exponential, Demo Sampling Percent: 0.25


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


...

Using CUDA...
Number of <state,action> pairs in the demonstrator data = 10000


Pre-training ...
All pre-train finish.
Annealing: linear, Demo Sampling Percent: 0.25
Episode: 0 Time Step: 38 Cumulative Rewards: 39.0
Annealing: linear, Demo Sampling Percent: 0.2475
Episode: 1 Time Step: 47 Cumulative Rewards: 48.0
Annealing: linear, Demo Sampling Percent: 0.245
Episode: 2 Time Step: 42 Cumulative Rewards: 43.0
Annealing: linear, Demo Sampling Percent: 0.2425
Episode: 3 Time Step: 52 Cumulative Rewards: 53.0
Annealing: linear, Demo Sampling Percent: 0.24
Episode: 4 Time Step: 27 Cumulative Rewards: 28.0
Annealing: linear, Demo Sampling Percent: 0.2375
Episode: 5 Time Step: 40 Cumulative Rewards: 41.0
Annealing: linear, Demo Sampling Percent: 0.235
Episode: 6 Time Step: 36 Cumulative Rewards: 37.0
Annealing: linear, Demo Sampling Percent: 0.23249999999999998
Episode: 7 Time Step: 25 Cumulative Rewards: 26.0
Annealing: linear, Demo Sampling Percent: 0.23
Episode: 8 Time Step: 38 Cumulative Rewards: 39.0
Annealing: linear, Demo Sampling Percent: 0.2275
Episode: 9 Time 

rewards,▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▂▁█▅▄▄▄▃▃▄▄▄▄▃▄▄▄▄▄▃▄▄▄▄
time_step,▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▂▁█▅▄▄▄▃▃▄▄▄▄▃▄▄▄▄▄▃▄▄▄▄
rewards,215.0
time_step,214


Using CUDA...
Number of <state,action> pairs in the demonstrator data = 10000


Pre-training ...
All pre-train finish.
Annealing: linear, Demo Sampling Percent: 0.5
Episode: 0 Time Step: 56 Cumulative Rewards: 57.0
Annealing: linear, Demo Sampling Percent: 0.49
Episode: 1 Time Step: 27 Cumulative Rewards: 28.0
Annealing: linear, Demo Sampling Percent: 0.48
Episode: 2 Time Step: 61 Cumulative Rewards: 62.0
Annealing: linear, Demo Sampling Percent: 0.47
Episode: 3 Time Step: 35 Cumulative Rewards: 36.0
Annealing: linear, Demo Sampling Percent: 0.46
Episode: 4 Time Step: 50 Cumulative Rewards: 51.0
Annealing: linear, Demo Sampling Percent: 0.45
Episode: 5 Time Step: 24 Cumulative Rewards: 25.0
Annealing: linear, Demo Sampling Percent: 0.44
Episode: 6 Time Step: 27 Cumulative Rewards: 28.0
Annealing: linear, Demo Sampling Percent: 0.43
Episode: 7 Time Step: 23 Cumulative Rewards: 24.0
Annealing: linear, Demo Sampling Percent: 0.42
Episode: 8 Time Step: 23 Cumulative Rewards: 24.0
Annealing: linear, Demo Sampling Percent: 0.41000000000000003
Episode: 9 Time Step: 25 Cu